In [ ]:
# %pip install streamlit

  Using cached streamlit-1.51.0-py3-none-any.whl.metadata (9.5 kB)
  Using cached altair-5.5.0-py3-none-any.whl.metadata (11 kB)
  Using cached blinker-1.9.0-py3-none-any.whl.metadata (1.6 kB)
  Using cached cachetools-6.2.2-py3-none-any.whl.metadata (5.6 kB)
  Using cached tenacity-9.1.2-py3-none-any.whl.metadata (1.2 kB)
  Using cached toml-0.10.2-py2.py3-none-any.whl.metadata (7.1 kB)
  Using cached watchdog-6.0.0-py3-none-win_amd64.whl.metadata (44 kB)
  Using cached gitpython-3.1.45-py3-none-any.whl.metadata (13 kB)
  Using cached pydeck-0.9.1-py2.py3-none-any.whl.metadata (4.1 kB)
  Using cached gitdb-4.0.12-py3-none-any.whl.metadata (1.2 kB)
  Using cached smmap-5.0.2-py3-none-any.whl.metadata (4.3 kB)
Using cached streamlit-1.51.0-py3-none-any.whl (10.2 MB)
Using cached altair-5.5.0-py3-none-any.whl (731 kB)
Using cached blinker-1.9.0-py3-none-any.whl (8.5 kB)
Using cached cachetools-6.2.2-py3-none-any.whl (11 kB)
Using cached gitpython-3.1.45-py3-none-any.whl (208 kB)
   -----


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


## 기본 코드

In [2]:
%%writefile app.py
import streamlit as st
import os
from openai import AzureOpenAI
from dotenv import load_dotenv
 
# 1. 환경 변수 로드 (.env 파일이 같은 폴더에 있어야 함)
load_dotenv()
 
st.title("🤖 나의 첫 AI 챗봇")
 
# 2. Azure OpenAI 클라이언트 설정
# (실제 값은 .env 파일이나 여기에 직접 입력하세요)
client = AzureOpenAI(
    api_key=os.getenv("AZURE_OAI_KEY"),
    api_version="2024-05-01-preview",
    azure_endpoint=os.getenv("AZURE_OAI_ENDPOINT")
)
 
# 3. 대화기록(Session State) 초기화 - 이게 없으면 새로고침 때마다 대화가 날아갑니다!
if "messages" not in st.session_state:
    st.session_state.messages = []
 
# 4. 화면에 기존 대화 내용 출력
for message in st.session_state.messages:
    with st.chat_message(message["role"]):
        st.markdown(message["content"])
 
# 5. 사용자 입력 받기
if prompt := st.chat_input("무엇을 도와드릴까요?"):
    # (1) 사용자 메시지 화면에 표시 & 저장
    st.chat_message("user").markdown(prompt)
    st.session_state.messages.append({"role": "user", "content": prompt})
 
    # (2) AI 응답 생성 (스트리밍 방식 아님, 단순 호출 예시)
    with st.chat_message("assistant"):
        response = client.chat.completions.create(
            model="gpt-4o-mini", # 사용하시는 배포명(Deployment Name)으로 수정 필요!
            messages=[
                {"role": m["role"], "content": m["content"]}
                for m in st.session_state.messages
            ]
        )
        assistant_reply = response.choices[0].message.content
        st.markdown(assistant_reply)
 
    # (3) AI 응답 저장
    st.session_state.messages.append({"role": "assistant", "content": assistant_reply})

Writing app.py


## 수정 코드

In [5]:
%%writefile app.py
import streamlit as st
import os
import json
import time
import requests
from openai import AzureOpenAI
from dotenv import load_dotenv
from zoneinfo import ZoneInfo
from datetime import datetime
from PIL import Image
import io

load_dotenv()

st.set_page_config(page_title="AI 챗봇", layout="centered")
st.title("통합 AI 챗봇")

# Azure OpenAI 클라이언트
client = AzureOpenAI(
    azure_endpoint=os.getenv("EXER_AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("EXER_AZURE_OPENAI_API_KEY"),
    api_version="2024-05-01-preview"
)

# === 실제로 사용하는 함수들 (기존 코드 그대로) ===
def get_current_weather(location, unit=None):
    location_lower = location.lower()
    cities = {"tokyo": "Tokyo", "san francisco": "San Francisco", "paris": "Paris", "seoul": "Seoul"}
    
    for key in cities:
        if key in location_lower:
            geo_url = f"https://geocoding-api.open-meteo.com/v1/search?name={key}&count=1"
            geo = requests.get(geo_url).json()
            if not geo.get("results"):
                return json.dumps({"location": location, "temperature": "unknown"})

            lat, lon = geo['results'][0]['latitude'], geo['results'][0]['longitude']
            weather_url = f"https://api.open-meteo.com/v1/forecast?latitude={lat}&longitude={lon}&current=temperature_2m,weather_code"
            data = requests.get(weather_url).json()

            return json.dumps({
                "location": cities[key],
                "temperature": data["current"]["temperature_2m"],
                "unit": "°C",
                "description": "맑음" if data["current"]["weather_code"] == 0 else "흐림/비 등"
            })
    return json.dumps({"location": location, "temperature": "unknown"})


def get_current_time(location):
    location_lower = location.lower()
    TIMEZONE_DATA = {
        "tokyo": "Asia/Tokyo",
        "san francisco": "America/Los_Angeles",
        "paris": "Europe/Paris",
        "seoul": "Asia/Seoul"
    }
    for key, tz in TIMEZONE_DATA.items():
        if key in location_lower:
            now = datetime.now(ZoneInfo(tz))
            return json.dumps({
                "location": key.capitalize(),
                "current_time": now.strftime("%Y년 %m월 %d일 %p %I:%M")
            })
    return json.dumps({"location": location, "current_time": "unknown"})


# === Assistant 한 번만 생성 (앱 시작 시 한 번만) ===
if "assistant" not in st.session_state:
    st.session_state.assistant = client.beta.assistants.create(
        name="날씨+그래프 봇",
        instructions="너는 친절한 한국어 챗봇이야. 날씨, 시간, 수식 그래프 그리기 등을 도와줘. 모르는 도시는 솔직히 모른다고 해.",
        model="gpt-4o-mini",  # 본인의 deployment name
        tools=[
            {"type": "code_interpreter"},
            {
                "type": "function",
                "function": {
                    "name": "get_current_weather",
                    "description": "주어진 도시의 현재 날씨를 반환",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "location": {"type": "string", "description": "도시 이름 (영어로 변환해서 전달)"},
                            "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]}
                        },
                        "required": ["location"]
                    }
                }
            },
            {
                "type": "function",
                "function": {
                    "name": "get_current_time",
                    "description": "주어진 도시의 현재 시간을 반환",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "location": {"type": "string", "description": "도시 이름 (영어로 변환해서 전달)"}
                        },
                        "required": ["location"]
                    }
                }
            }
        ]
    )

# === 대화 세션별 thread 관리 ===
if "thread_id" not in st.session_state:
    thread = client.beta.threads.create()
    st.session_state.thread_id = thread.id

# === 과거 대화 표시 ===
if "messages" not in st.session_state:
    st.session_state.messages = []

for msg in st.session_state.messages:
    with st.chat_message(msg["role"]):
        st.markdown(msg["content"])
        if "images" in msg:
            for img_bytes in msg["images"]:
                st.image(img_bytes, width=500)

# === 사용자 입력 ===
if prompt := st.chat_input("궁금한 거 물어보세요!"):
    # 1. 사용자 메시지 추가
    st.session_state.messages.append({"role": "user", "content": prompt})
    with st.chat_message("user"):
        st.markdown(prompt)

    # 2. Thread에 메시지 추가
    client.beta.threads.messages.create(
        thread_id=st.session_state.thread_id,
        role="user",
        content=prompt
    )

    # 3. Run 시작
    with st.chat_message("assistant"):
        message_placeholder = st.empty()
        full_response = ""
        
        run = client.beta.threads.runs.create(
            thread_id=st.session_state.thread_id,
            assistant_id=st.session_state.assistant.id
        )

        # Run 상태 polling
        while run.status in ["queued", "in_progress", "requires_action"]:
            time.sleep(0.5)
            run = client.beta.threads.runs.retrieve(
                thread_id=st.session_state.thread_id,
                run_id=run.id
            )

            # tool 호출이 필요한 경우
            if run.status == "requires_action":
                tool_outputs = []
                for tool_call in run.required_action.submit_tool_outputs.tool_calls:
                    if tool_call.function.name == "get_current_weather":
                        args = json.loads(tool_call.function.arguments)
                        output = get_current_weather(**args)
                        tool_outputs.append({"tool_call_id": tool_call.id, "output": output})
                    elif tool_call.function.name == "get_current_time":
                        args = json.loads(tool_call.function.arguments)
                        output = get_current_time(**args)
                        tool_outputs.append({"tool_call_id": tool_call.id, "output": output})

                # tool 결과 제출
                run = client.beta.threads.runs.submit_tool_outputs(
                    thread_id=st.session_state.thread_id,
                    run_id=run.id,
                    tool_outputs=tool_outputs
                )

        # 최종 완료
        if run.status == "completed":
            messages = client.beta.threads.messages.list(thread_id=st.session_state.thread_id)
            latest_message = messages.data[0]

            response_text = ""
            image_bytes_list = []

            for block in latest_message.content:
                if block.type == "text":
                    response_text += block.text.value
                elif block.type == "image_file":
                    file_id = block.image_file.file_id
                    image_data = client.files.content(file_id).read()
                    image = Image.open(io.BytesIO(image_data))
                    buffered = io.BytesIO()
                    image.save(buffered, format="PNG")
                    image_bytes_list.append(buffered.getvalue())

            # 화면에 출력
            full_response = response_text
            message_placeholder.markdown(full_response + "▌")

            # 이미지 있으면 표시
            for img_bytes in image_bytes_list:
                st.image(img_bytes, width=500)

            # 세션에 저장
            st.session_state.messages.append({
                "role": "assistant",
                "content": full_response,
                "images": image_bytes_list
            })

Overwriting app.py
